In [275]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [276]:
matches1 = pd.read_csv("assets/2022-2023.csv")
matches2 = pd.read_csv("assets/2023-2024.csv")
matches3 = pd.read_csv("assets/2024-2025.csv")
matches = pd.concat([matches1, matches2, matches3])

In [277]:
matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])

""" matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])
matches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) """

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1030/1605438902.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])


' matches["Date (US Eastern)"] = pd.to_datetime(matches["Date (US Eastern)"])\nmatches["Time (US Eastern)"] = pd.to_datetime(matches["Time (US Eastern)"]) '

In [278]:
#advantages and disadvantages to consider when teams play against each other

matches["homeAway_code"] = np.where(matches["Team"] == matches["Home Team"], 0, 1) 
#matches["homeAway"].astype("category").cat.codes

matches["opponent_code"] = np.where(
    matches["Team"] == matches["Home Team"],
    matches["Away Team"].astype("category").cat.codes,
    matches["Home Team"].astype("category").cat.codes
)
#matches["hour"] = matches["Date Time (US Eastern)"].dt.hour.astype("Int64")
matches["day_code"] = matches["Date Time (US Eastern)"].dt.dayofweek
matches["date"] = matches["Date Time (US Eastern)"].dt.date

In [279]:
#determine the match result
matches["target"] = np.where(
    matches["Home Goal"] == matches["Away Goal"], 2,  
    np.where(
        (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] == matches["Home Team"]), 1,  
        np.where(
            (matches["Home Goal"] > matches["Away Goal"]) & (matches["Team"] != matches["Home Team"]), 0,  
            np.where(
                (matches["Home Goal"] < matches["Away Goal"]) & (matches["Team"] == matches["Away Team"]), 1,  
                0  
            )
        )
    )
)

In [280]:
#began using machine learning 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [281]:
""" predictors = [   
    "homeAway_code", "opponent_code", "day_code", 
    "shotsOnTarget", "accuratePasses", "accurateCrosses", 
    "accurateLongBalls", "blockedShots", "effectiveTackles", 
    "effectiveClearance","foulsCommitted", "saves"
    ] """
#predictors = ["homeAway_code", "opponent_code", "day_code"]

' predictors = [   \n    "homeAway_code", "opponent_code", "day_code", \n    "shotsOnTarget", "accuratePasses", "accurateCrosses", \n    "accurateLongBalls", "blockedShots", "effectiveTackles", \n    "effectiveClearance","foulsCommitted", "saves"\n    ] '

In [282]:
#train = matches[matches["Date Time (US Eastern)"] <= '2024-05-01']

In [283]:
#test = matches[matches["Date Time (US Eastern)"] > '2024-05-01']

In [284]:
#rf.fit(train[predictors], train["target"])

In [285]:
#preds = rf.predict(test[predictors])

In [286]:
""" accuracy = accuracy_score(test["target"], preds)
accuracy """

' accuracy = accuracy_score(test["target"], preds)\naccuracy '

In [287]:
#combine = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [288]:
#pd.crosstab(index=combine["actual"], columns=combine["prediction"])

In [289]:
#precision_score(test["target"], preds, average='weighted', zero_division=0)

In [290]:
#grouped_matches = matches.groupby("Team")
""" group = grouped_matches.get_group("West Ham United")
group """

' group = grouped_matches.get_group("West Ham United")\ngroup '

In [291]:
""" 
This function uses the stats from the previous 3 games 
to determine the team's condition and predict the next game. 
"""
columns = ["shotsOnTarget", "accuratePasses", "accurateCrosses", 
    "accurateLongBalls", "blockedShots", "effectiveTackles", 
    "effectiveClearance","foulsCommitted", "saves"]
new_columns = [f"{c}_rolling" for c in columns]

def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date Time (US Eastern)")
    rolling_stats = group[cols].shift(1).rolling(window=3).mean()
    for col, new_col in zip(cols, new_cols):
            group[new_col] = rolling_stats[col]
    group = group.dropna(subset=new_cols)
    return group

In [292]:
""" rolling_averages(group, columns, new_columns) """
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))
matches_rolling.droplevel("Team")
matches_rolling = matches_rolling.reset_index(drop=True)
matches_rolling.index = range(matches_rolling.shape[0])
#matches_rolling

/var/folders/cn/j21vzthj6ds5gjqdtwlgj4f00000gn/T/ipykernel_1030/1544277950.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, columns, new_columns))


In [293]:

# Define predictors
predictors = ["homeAway_code", "opponent_code", "day_code"] + new_columns

# Train-Test Split
train = matches_rolling[matches_rolling["Date Time (US Eastern)"] <= '2024-05-01']
test = matches_rolling[matches_rolling["Date Time (US Eastern)"] > '2024-05-01']

# Train the Model
rf.fit(train[predictors], train["target"])

# Make predictions
preds = rf.predict(test[predictors])
accuracy = accuracy_score(test["target"], preds)
accuracy

0.48771929824561405

In [294]:
def make_predictions(data, predictors):
    train = data[data["Date Time (US Eastern)"] <= '2024-05-01']
    test = data[data["Date Time (US Eastern)"] > '2024-05-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combine = pd.DataFrame(dict(actual=test["target"], prediction=preds), index=test.index)
    precision = precision_score(test["target"], preds, average='macro', zero_division=0)
    return combine, precision

In [295]:
combined, precision = make_predictions(matches_rolling, predictors + new_columns)
combined = combined.merge(matches_rolling[["date", "Team" ,"Home Team", "Away Team", "target"]], left_index=True, right_index=True)

combined_liverpool = combined[combined["Team"] == "Chelsea"]
combined_liverpool



,actual,prediction,date,Team,Home Team,Away Team,target
523,1,1,2024-05-02,Chelsea,Chelsea,Tottenham Hotspur,1
524,1,1,2024-05-05,Chelsea,Chelsea,West Ham United,1
525,1,1,2024-05-11,Chelsea,Nottingham Forest,Chelsea,1
526,1,1,2024-05-15,Chelsea,Brighton & Hove Albion,Chelsea,1
527,1,1,2024-05-19,Chelsea,Chelsea,AFC Bournemouth,1
528,0,1,2024-08-18,Chelsea,Chelsea,Manchester City,0
529,1,1,2024-08-25,Chelsea,Wolverhampton Wanderers,Chelsea,1
530,2,2,2024-09-01,Chelsea,Chelsea,Crystal Palace,2
531,1,0,2024-09-14,Chelsea,AFC Bournemouth,Chelsea,1
532,1,1,2024-09-21,Chelsea,West Ham United,Chelsea,1


In [300]:
def predict_next_match(home_team, away_team, date_time):

    if home_team not in matches["Home Team"].unique() or away_team not in matches["Home Team"].unique():
        print("error")
        return
    
    day_code = pd.to_datetime(date_time).dayofweek

    homeAway_code = 0 if home_team == home_team else 1
    opponent_code = matches["Home Team"].astype("category").cat.categories.get_loc(away_team)

    rolling_values = [0] * len(new_columns) 
    match_data = pd.DataFrame([{
        "homeAway_code": homeAway_code,
        "opponent_code": opponent_code,
        "day_code": day_code,
        **{col: val for col, val in zip(new_columns, rolling_values)}
    }])

    match_data = match_data[predictors]
    prediction = rf.predict(match_data)[0]

    result = "Win" if prediction[0] == 1 else "Loss" if prediction[0] == 0 else "Draw"

    print(f"Prediction for {home_team} vs {away_team}: {result} for {home_team}")
        
    


In [301]:
predict_next_match("Chelsea", "Newcastle United", "2024-06-01 15:00")


ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.
